# Tool DSL Parsing

In [11]:
import weaviate
from weaviate.classes.query import Filter
from weaviate.classes.query import Metrics # used for aggregation filter (not in this code cell block)
from weaviate.classes.aggregate import GroupByAggregate # used for aggregation filter (not in this code cell block)

def _build_weaviate_filter(filter_string: str) -> Filter:
    def _parse_condition(condition: str) -> Filter:
        parts = condition.split(':')
        if len(parts) < 3:
            raise ValueError(f"Invalid condition: {condition}")

        property, operator, value = parts[0], parts[1], ':'.join(parts[2:])

        if operator == '=':
            return Filter.by_property(property).equal(value)
        elif operator == '!=':
            return Filter.by_property(property).not_equal(value)
        elif operator == '>':
            return Filter.by_property(property).greater_than(float(value))
        elif operator == '<':
            return Filter.by_property(property).less_than(float(value))
        elif operator == '>=':
            return Filter.by_property(property).greater_than_equal(float(value))
        elif operator == '<=':
            return Filter.by_property(property).less_than_equal(float(value))
        elif operator == 'LIKE':
            return Filter.by_property(property).like(value)
        elif operator == 'CONTAINS_ANY':
            return Filter.by_property(property).contains_any(value.split(','))
        elif operator == 'CONTAINS_ALL':
            return Filter.by_property(property).contains_all(value.split(','))
        elif operator == 'WITHIN':
            lat, lon, dist = map(float, value.split(','))
            return Filter.by_property(property).within_geo_range(lat, lon, dist)
        else:
            raise ValueError(f"Unsupported operator: {operator}")

    def _parse_group(group: str) -> Filter:
        if 'AND' in group:
            conditions = [_parse_group(g.strip()) for g in group.split('AND')]
            return Filter.all_of(conditions)
        elif 'OR' in group:
            conditions = [_parse_group(g.strip()) for g in group.split('OR')]
            return Filter.any_of(conditions)
        else:
            return _parse_condition(group)

    # Remove outer parentheses if present
    filter_string = filter_string.strip()
    if filter_string.startswith('(') and filter_string.endswith(')'):
        filter_string = filter_string[1:-1]

    return _parse_group(filter_string)

In [12]:
_build_weaviate_filter("name:=:charles")

_FilterValue(value='charles', operator=<_Operator.EQUAL: 'Equal'>, target='name')

In [13]:
from typing import Tuple, Optional, List
from weaviate.classes.query import Metrics
from weaviate.classes.aggregate import GroupByAggregate
import re

def _build_weaviate_aggregation(agg_string: str) -> Tuple[Optional[GroupByAggregate], List[Metrics]]:
    """
    Parses an aggregation string into Weaviate GroupByAggregate and Metrics objects.
    
    Format:
    GROUP_BY(property) METRICS(property:type[metrics], property2:type[metrics])
    
    Examples:
    - "GROUP_BY(publication) METRICS(wordCount:int[count,mean,max])"
    - "METRICS(rating:num[mean,sum], title:text[count,topOccurrences])"
    - "GROUP_BY(author) METRICS(isPublished:bool[totalTrue,percentageFalse])"
    
    Returns:
    Tuple of (GroupByAggregate, List[Metrics])
    """
    def _parse_metrics(metrics_str: str) -> List[Metrics]:
        # Extract content within METRICS(...)
        metrics_list = []
        
        # Split multiple metrics definitions by comma, but not within brackets
        metrics_parts = []
        current_part = ""
        bracket_count = 0
        
        for char in metrics_str:
            if char == '[':
                bracket_count += 1
            elif char == ']':
                bracket_count -= 1
            elif char == ',' and bracket_count == 0:
                metrics_parts.append(current_part.strip())
                current_part = ""
                continue
            current_part += char
        if current_part:
            metrics_parts.append(current_part.strip())
        
        # Parse each metric definition
        for metric in metrics_parts:
            # Parse property:type[operations]
            match = re.match(r'(\w+):(text|int|num|bool)\[([\w,]+)\]', metric.strip())
            if not match:
                raise ValueError(f"Invalid metrics format: {metric}")
            
            prop_name, data_type, operations = match.groups()
            operations = [op.strip() for op in operations.split(',')]
            
            # Create appropriate Metrics object based on type
            if data_type == 'text':
                metric_obj = Metrics(prop_name).text(
                    count='count' in operations,
                    top_occurrences='topOccurrences' in operations
                )
            elif data_type in ('int', 'num'):
                metric_obj = Metrics(prop_name).number(
                    count='count' in operations,
                    minimum='min' in operations,
                    maximum='max' in operations,
                    mean='mean' in operations,
                    median='median' in operations,
                    mode='mode' in operations,
                    sum_='sum' in operations
                )
            elif data_type == 'bool':
                metric_obj = Metrics(prop_name).boolean(
                    count='count' in operations,
                    total_true='totalTrue' in operations,
                    total_false='totalFalse' in operations,
                    percentage_true='percentageTrue' in operations,
                    percentage_false='percentageFalse' in operations
                )
            else:
                raise ValueError(f"Unsupported data type: {data_type}")
                
            metrics_list.append(metric_obj)
        
        return metrics_list

    def _parse_group_by(group_str: str) -> GroupByAggregate:
        # Extract property name from GROUP_BY(property)
        match = re.match(r'GROUP_BY\((\w+)\)', group_str)
        if not match:
            raise ValueError(f"Invalid GROUP_BY format: {group_str}")
        
        return GroupByAggregate(prop=match.group(1))

    # Initialize return values
    group_by = None
    metrics = []
    
    # Split into parts by space, but not within parentheses
    parts = []
    current_part = ""
    paren_count = 0
    
    for char in agg_string:
        if char == '(':
            paren_count += 1
        elif char == ')':
            paren_count -= 1
        elif char.isspace() and paren_count == 0:
            if current_part:
                parts.append(current_part)
            current_part = ""
            continue
        current_part += char
    if current_part:
        parts.append(current_part)
    
    # Parse each part
    for part in parts:
        if part.startswith('GROUP_BY'):
            if group_by is not None:
                raise ValueError("Multiple GROUP_BY clauses not allowed")
            group_by = _parse_group_by(part)
        elif part.startswith('METRICS'):
            # Extract content within METRICS(...)
            match = re.match(r'METRICS\((.*)\)', part)
            if not match:
                raise ValueError(f"Invalid METRICS format: {part}")
            metrics.extend(_parse_metrics(match.group(1)))
    
    return group_by, metrics

In [14]:
agg_str1 = "METRICS(wordCount:int[count,mean,max])"
group_by1, metrics1 = _build_weaviate_aggregation(agg_str1)

TypeError: Metrics.number() got an unexpected keyword argument 'type'